In [93]:
import pandas as pd
import re

In [137]:
import pandas as pd
import re

def kilimall_phones_clean(csv_path):
    #import data
    dataframe = pd.read_csv(csv_path)
    dataframe.drop(columns = 'Unnamed: 0', inplace = True)
    dataframe = dataframe.drop_duplicates()
    dataframe['price']= dataframe['phones_price'].str.strip('KSh,').str.replace(",","")
    dataframe['number_of_reviews'] = dataframe['phones_reviews'].str.extract(r'(\d+)').astype(int)
    # List of words to remove
    words_to_remove = [
    'xmas', 'sale', 'new','New Arrival', 'Xmas','Promotion','Year','phones','arrival', 'brand', 'refurbished', 'year', 'sales', 'promotion', 
    'original', 'super', 'limited', 'offer', 'holiday', 'offers', 'black', 'friday', 'deals', 
    'arrivals', 'christmas', 'smart', 'end', 'free', 'charger', 'official', 'phones', 'special', 'Smart',
    'buy', 'big', 'launch', 'november','Limited','OFFER','Brand','New','Face','unlock','Arrival','Black','Friday','NEW','CHEAP','SMART','BLACK','NOVEMBER','OFFERS','Unlocked','Phones','BRAND','Original','Free','Charger','Exclusive','Special','Offer','Phone','LIMITED'
    'anniversary','Arrivals','Best','for','a','Refirbishe','time','BRANDNEW','CHRISTMAS','HOLIDAY','OFFEROFFER','REFURBISHED','END','YEAR','Buy','Official','Offiicial','BIG','DEALS','ARRIVALS','OFFERS','Unlock','Google','system','Techweek','mobile','Harambee','phone','Global','Version'
    ,'Refurbishe','Systems','Edition','Honor','ORIGINAL','Freeyond','machine','The','Refur','version','Latest','SOWHAT'
    ,'GRAB','QUICK','SALE','Firmware','Super','Sale','Badili','FreeYond','Smartphone','HONOR','Cell','net','Top','International','edition','high','matching','EASTER','Sharp','YOUR'
    ,'certified','Certified','DISCOUNT','SPECIAL','SPECIAL','ARRIVAL','Renovate','SEALED','OFFERSBRAND','EXPANDABLE','FRIDAY','FLASH','LIMTED'
    ]
    pattern = r'\b(?:' + '|'.join(map(re.escape, words_to_remove)) + r')\b'

    dataframe['phones_name'] = dataframe['phones_name'].str.replace(pattern, '', regex=True).str.strip()


    # Functions to clean the data
    def remove_brackets(column):
        return column.str.replace(r'\[.*?\]', '', regex=True)

    def clean_column(column):
        return column.str.replace(pattern, '', flags=re.IGNORECASE).str.replace(r'\s+', ' ', regex=True).str.strip()

    def remove_symbols(column):
        return column.str.replace(r'[!"+]', '', regex=True)

    # Apply cleaning functions
    dataframe['phones_name'] = remove_brackets(dataframe['phones_name'])  # Remove text in square brackets
    dataframe['phones_name'] = clean_column(dataframe['phones_name'])    # Remove unwanted words
    dataframe['phones_name'] = remove_symbols(dataframe['phones_name'])  # Remove special characters
    dataframe["phones_name"] = dataframe["phones_name"].str.replace("Refurbished", "")

    def clean_phones_name(name):
    # Remove unwanted characters: (), {}, !!, [], "", and numbers before the first letter
        name = re.sub(r'[\(\)\{\}\[\]\"!]+', '', name)  # Remove parentheses, brackets, quotes, and exclamation marks
        name = re.sub(r'^\d+\s*', '', name)  # Remove numbers at the start of the string
        return name.strip()  # Remove leading and trailing whitespace

    # Apply the cleaning function to the phones_name column
    dataframe["phones_name"] = dataframe["phones_name"].apply(clean_phones_name)

    def remove_non_alphabet_start(name):
        return re.sub(r'^[^a-zA-Z]+', '', name)  # Remove any non-alphabet characters from the start

    # Apply the function to the phones_name column
    dataframe["phones_name"] = dataframe["phones_name"].apply(remove_non_alphabet_start)


    ram_pattern = r'\b(2|3|4|6|8|12|24)\s*GB'

    # Function to extract RAM size
    def extract_ram(phones_name):
        ram = re.findall(ram_pattern, phones_name)  # Find all occurrences that match the pattern
        return ram[0] if ram else None  # If found, return the first match; else return None

    storage_pattern = r'\b(16|32|64|128|256)\s*GB'
    def extract_storage(phones_name):
        storage = re.findall(storage_pattern, phones_name)  # Find all occurrences that match the pattern
        return storage[0] if storage else None  # If found, return the first match; else return None
    def extract_brand(phones_name):
        # Split the name into words and take the first two or three
        words = phones_name.split()
        return ' '.join(words[:3])  # Return the first three words (or fewer if not available)
    def extract_battery(phones_name):
        # Search for a pattern that captures numbers followed by 'mAh'
        match = re.search(r'(\d+)\s*mAh', phones_name)
        if match:
            return match.group(1)  # Return the first captured group (the number)
        return None  # If no match, return None

    # Apply the function to the 'phones_name' column and create a new 'battery' column
    dataframe['battery'] = dataframe['phones_name'].apply(extract_battery)
    # Apply the function to the 'phones_name' column and create a new 'storage' column
    dataframe['ram'] = dataframe['phones_name'].apply(extract_ram)
    dataframe['storage'] = dataframe['phones_name'].apply(extract_storage)
    # Apply the function to the 'phones_name' column and create a new 'brand' column
    dataframe['brand'] = dataframe['phones_name'].apply(extract_brand)
    dataframe["description"] = dataframe['phones_name']
    dataframe['source'] = ['kilimall'] *len(dataframe)
    dataframe['id'] = [i for i in range(1,len(dataframe)+1)]
    dataframe['urls'] = dataframe['phone_url']
    dataframe = dataframe.drop(columns = ['phones_name', 'phones_reviews', 'phones_price', 'phone_url'])
    # dataframe.rename({""})
    columns = ["id", "description", "brand", "price", "number_of_reviews", "ram", "storage", "battery", "source", "urls"]
    dataframe = dataframe[columns]
    dataframe.to_csv("../data/clean/kilimall_clean_phones.csv", index = False)


    

    return dataframe





csv_path =(r"C:\Users\Vivian.Obino1\Desktop\e-commerce analysis\data\scraped\kilimall_phones_scraped.csv")
dataframe = kilimall_phones_clean(csv_path)
dataframe

,id,description,brand,price,number_of_reviews,ram,storage,battery,source,urls
0,1,Itel S23 Plus 128GB 844GB RAM 6.78'' Big Curv...,Itel S23 Plus,13599,178,None,128,5000,kilimall,https://www.kilimall.co.ke/listing/3000123-ite...
1,2,OPPO F11 A9 128GB6GB 6.5 4000mAh Fingerprint D...,OPPO F11 A9,7499,24,None,128,4000,kilimall,https://www.kilimall.co.ke/listing/1000377595-...
2,3,XIAOMI Redmi 14C 128GB Storage Upto 844GB Ram ...,XIAOMI Redmi 14C,12910,157,None,128,5160,kilimall,https://www.kilimall.co.ke/listing/1000332494-...
3,4,TECNO POP 9 128GB 633GB 4.5G Fast Network 6.6...,TECNO POP 9,11499,95,None,128,5000,kilimall,https://www.kilimall.co.ke/listing/1000332146-...
4,5,Itel A70 128GB835GB RAM 6.6 Screen 4G Network ...,Itel A70 128GB835GB,9250,582,None,128,5000,kilimall,https://www.kilimall.co.ke/listing/2988603-ite...
...,...,...,...,...,...,...,...,...,...,...
5359,5353,"Samsung Galaxy A16, 6.5''HD, 6GB RAM 128GB RO...","Samsung Galaxy A16,",24000,0,6,128,5000,kilimall,https://www.kilimall.co.ke/listing/1001011301-...
5360,5354,"Samsung Galaxy A16, 6.5''HD, 6GB RAM 128GB RO...","Samsung Galaxy A16,",24500,0,6,128,5000,kilimall,https://www.kilimall.co.ke/listing/1001011283-...
5361,5355,VILLAON V20 SE 6.3 2GB RAM32GB Storage 8MP Cam...,VILLAON V20 SE,4999,0,2,None,3000,kilimall,https://www.kilimall.co.ke/listing/1001010193-...
5362,5356,"XIAOMI Redmi Note 13, 6.67'', 8GBRAM 256GBROM,...",XIAOMI Redmi Note,21500,0,8,256,None,kilimall,https://www.kilimall.co.ke/listing/1001000979-...


In [121]:
dataframe.columns

Index(['phones_reviews', 'phones_price', 'phone_url', 'battery', 'ram',
       'storage', 'brand', 'descriptions', 'source', 'id', 'urls'],
      dtype='object')

In [100]:
dataframe = pd.read_csv(r"C:\Users\Vivian.Obino1\Desktop\e-commerce analysis\data\scraped\kilimall_phones_scraped.csv")
dataframe.head()

,Unnamed: 0,phones_name,phones_reviews,phones_price,phone_url
0,0,Itel S23 Plus 128GB + 8(4+4)GB RAM 6.78'' Big ...,(178),"KSh 13,599",https://www.kilimall.co.ke/listing/3000123-ite...
1,1,Refurbished OPPO F11 A9 Smart Phone 128GB+6GB ...,(24),"KSh 7,499",https://www.kilimall.co.ke/listing/1000377595-...
2,2,XIAOMI Redmi 14C 128GB Storage Upto 8(4+4)GB R...,(157),"KSh 12,910",https://www.kilimall.co.ke/listing/1000332494-...
3,3,New Arrival TECNO POP 9 128GB + 6(3+3)GB 4.5G ...,(95),"KSh 11,499",https://www.kilimall.co.ke/listing/1000332146-...
4,4,"Itel A70 128GB+8(3+5)GB RAM 6.6"" Screen 4G Ne...",(582),"KSh 9,250",https://www.kilimall.co.ke/listing/2988603-ite...


In [101]:
# print(dataframe)
dataframe.drop(columns = 'Unnamed: 0', inplace = True)
dataframe

,phones_name,phones_reviews,phones_price,phone_url
0,Itel S23 Plus 128GB + 8(4+4)GB RAM 6.78'' Big ...,(178),"KSh 13,599",https://www.kilimall.co.ke/listing/3000123-ite...
1,Refurbished OPPO F11 A9 Smart Phone 128GB+6GB ...,(24),"KSh 7,499",https://www.kilimall.co.ke/listing/1000377595-...
2,XIAOMI Redmi 14C 128GB Storage Upto 8(4+4)GB R...,(157),"KSh 12,910",https://www.kilimall.co.ke/listing/1000332494-...
3,New Arrival TECNO POP 9 128GB + 6(3+3)GB 4.5G ...,(95),"KSh 11,499",https://www.kilimall.co.ke/listing/1000332146-...
4,"Itel A70 128GB+8(3+5)GB RAM 6.6"" Screen 4G Ne...",(582),"KSh 9,250",https://www.kilimall.co.ke/listing/2988603-ite...
...,...,...,...,...
5359,"Samsung Galaxy A16, 6.5''HD+, 6GB RAM + 128GB ...",(0),"KSh 24,000",https://www.kilimall.co.ke/listing/1001011301-...
5360,"Samsung Galaxy A16, 6.5''HD+, 6GB RAM + 128GB ...",(0),"KSh 24,500",https://www.kilimall.co.ke/listing/1001011283-...
5361,"VILLAON V20 SE 6.3"" 2GB RAM+32GB Storage 8MP C...",(0),"KSh 4,999",https://www.kilimall.co.ke/listing/1001010193-...
5362,"XIAOMI Redmi Note 13, 6.67'', 8GBRAM+ 256GBROM...",(0),"KSh 21,500",https://www.kilimall.co.ke/listing/1001000979-...


In [82]:
dataframe_cleaned = dataframe.drop_duplicates()
print(dataframe)


                                            phones_name phones_reviews  \
0     Itel S23 Plus 128GB + 8(4+4)GB RAM 6.78'' Big ...          (178)   
1     Refurbished OPPO F11 A9 Smart Phone 128GB+6GB ...           (24)   
2     XIAOMI Redmi 14C 128GB Storage Upto 8(4+4)GB R...          (157)   
3     New Arrival TECNO POP 9 128GB + 6(3+3)GB 4.5G ...           (95)   
4     Itel A70 128GB+8(3+5)GB RAM 6.6" Screen  4G Ne...          (582)   
...                                                 ...            ...   
5359  Samsung Galaxy A16, 6.5''HD+, 6GB RAM + 128GB ...            (0)   
5360  Samsung Galaxy A16, 6.5''HD+, 6GB RAM + 128GB ...            (0)   
5361  VILLAON V20 SE 6.3" 2GB RAM+32GB Storage 8MP C...            (0)   
5362  XIAOMI Redmi Note 13, 6.67'', 8GBRAM+ 256GBROM...            (0)   
5363  GALAXY A06,6GB RAM & 128GB ROM, & 5000MAH,DUAL...            (0)   

     phones_price                                          phone_url  
0      KSh 13,599  https://www.kilimall.

In [84]:
dataframe['phones_price']= dataframe['phones_price'].str.strip('KSh,').str.replace(",","")
dataframe

,phones_name,phones_reviews,phones_price,phone_url
0,Itel S23 Plus 128GB + 8(4+4)GB RAM 6.78'' Big ...,(178),13599,https://www.kilimall.co.ke/listing/3000123-ite...
1,Refurbished OPPO F11 A9 Smart Phone 128GB+6GB ...,(24),7499,https://www.kilimall.co.ke/listing/1000377595-...
2,XIAOMI Redmi 14C 128GB Storage Upto 8(4+4)GB R...,(157),12910,https://www.kilimall.co.ke/listing/1000332494-...
3,New Arrival TECNO POP 9 128GB + 6(3+3)GB 4.5G ...,(95),11499,https://www.kilimall.co.ke/listing/1000332146-...
4,"Itel A70 128GB+8(3+5)GB RAM 6.6"" Screen 4G Ne...",(582),9250,https://www.kilimall.co.ke/listing/2988603-ite...
...,...,...,...,...
5359,"Samsung Galaxy A16, 6.5''HD+, 6GB RAM + 128GB ...",(0),24000,https://www.kilimall.co.ke/listing/1001011301-...
5360,"Samsung Galaxy A16, 6.5''HD+, 6GB RAM + 128GB ...",(0),24500,https://www.kilimall.co.ke/listing/1001011283-...
5361,"VILLAON V20 SE 6.3"" 2GB RAM+32GB Storage 8MP C...",(0),4999,https://www.kilimall.co.ke/listing/1001010193-...
5362,"XIAOMI Redmi Note 13, 6.67'', 8GBRAM+ 256GBROM...",(0),21500,https://www.kilimall.co.ke/listing/1001000979-...


In [92]:
dataframe['phones_reviews'] = dataframe['phones_reviews'].str.extract(r'(\d+)').astype(int)
dataframe

KeyError: 'phones_reviews'

In [91]:
dataframe.to_csv("kilimall_clean_phones.csv", index=False)

In [90]:
import re
import pandas as pd

# Replace this with the path to your actual CSV file
file_path = 'kilimall_clean_phones.csv'

# Load the CSV file
dataframe = pd.read_csv(file_path, header=None)

# List of words to remove
words_to_remove = [
    'xmas', 'sale', 'new', 'arrival', 'brand', 'refurbished', 'year', 'sales', 'promotion', 
    'original', 'super', 'limited', 'offer', 'holiday', 'offers', 'black', 'friday', 'deals', 
    'arrivals', 'christmas', 'smart', 'end', 'free', 'charger', 'official', 'phones', 'special', 
    'buy', 'big', 'launch', 'november'
]

# Compile regex pattern
pattern = r'\b(?:' + '|'.join(map(re.escape, words_to_remove)) + r')\b'

# Functions to clean the data
def remove_brackets(column):
    return column.str.replace(r'\[.*?\]', '', regex=True)

def clean_column(column):
    return column.str.replace(pattern, '', flags=re.IGNORECASE).str.replace(r'\s+', ' ', regex=True).str.strip()

def remove_symbols(column):
    return column.str.replace(r'[!"+]', '', regex=True)

# Apply cleaning functions
dataframe['phones_name'] = remove_brackets(dataframe['phones_name'])  # Remove text in square brackets
dataframe['phones_name'] = clean_column(dataframe['phones_name'])    # Remove unwanted words
dataframe['phones_name'] = remove_symbols(dataframe['phones_name'])  # Remove special characters

# Save cleaned data to a new CSV file
cleaned_file_path = 'kilimall_clean_phones.csv'
dataframe.to_csv(cleaned_file_path, index=False)

# Display a sample of the cleaned data
print(dataframe.head())


KeyError: 'phones_name'

In [60]:
# import re
# pattern = r"[Refurbished]"
dataframe["phones_name"] = dataframe["phones_name"].str.replace("Refurbished", "")
dataframe.head()

,,phones_name,unknown_col,phones_price
Unnamed: 0,phones_name,phones_reviews,phones_price,phone_url
0,Itel S23 Plus 128GB + 8(4+4)GB RAM 6.78'' Big Curved Screen Dual SIM 4G Phone 50MP+32MP AI Camera 5000mAh battery 18W Charging Under Display Fingerprint With NFC S23+ Smart Phones,178,"13,599",https://www.kilimall.co.ke/listing/3000123-ite...
1,"Refurbished OPPO F11 A9 Smart Phone 128GB+6GB 6.5"" 4000mAh Fingerprint unlock phone Dual SIM 4G phone Refurbished Phone SmartPhones Mobile Phones 128GB phone 4G Phone oppp phone smartphone",24,"7,499",https://www.kilimall.co.ke/listing/1000377595-...
2,"XIAOMI Redmi 14C 128GB Storage Upto 8(4+4)GB Ram 50MP AI Dual Camera Immersive 6.88"" UHD Display Upto 120Hz Refresh Rate Dual Sim 5160mAh (typ) Battery Elegant glass back cover Smart Phone",157,"12,910",https://www.kilimall.co.ke/listing/1000332494-...
3,"New Arrival TECNO POP 9 128GB + 6(3+3)GB 4.5G Fast Network 6.6"" 120Hz Dual Speakers IP54 Waterproof 13MP 5000mAh Type-C Android 14 Dual SIM Smart Phones Side Fingerprint phone",95,"11,499",https://www.kilimall.co.ke/listing/1000332146-...


In [61]:
dataframe.to_csv("kilimall_clean_phones.csv", index=False)


In [62]:

# List of words to remove
words_to_remove = [
    'xmas', 'sale', 'new', 'Xmas','Promotion','Year','phones','arrival', 'brand', 'refurbished', 'year', 'sales', 'promotion', 
    'original', 'super', 'limited', 'offer', 'holiday', 'offers', 'black', 'friday', 'deals', 
    'arrivals', 'christmas', 'smart', 'end', 'free', 'charger', 'official', 'phones', 'special', 'Smart',
    'buy', 'big', 'launch', 'november','Limited','OFFER','Brand','New','Face','unlock','Arrival','Black','Friday','NEW','CHEAP','SMART','BLACK','NOVEMBER','OFFERS','Unlocked','Phones','BRAND','Original','Free','Charger','Exclusive','Special','Offer','Phone','LIMITED'
    'anniversary','Arrivals','Best','for','a','Refirbishe','time','BRANDNEW','CHRISTMAS','HOLIDAY','OFFEROFFER','REFURBISHED','END','YEAR','Buy','Official','Offiicial','BIG','DEALS','ARRIVALS','OFFERS','Unlock','Google','system','Techweek','mobile','Harambee','phone','Global','Version'
    ,'Refurbishe','Systems','Edition','Honor','ORIGINAL','Freeyond','machine','The','Refur','version','Latest','SOWHAT'
,'GRAB','QUICK','SALE','Firmware','Super','Sale','Badili','FreeYond','Smartphone','HONOR','Cell','net','Top','International','edition','high','matching','EASTER','Sharp','YOUR'
,'certified','Certified','DISCOUNT','SPECIAL','SPECIAL','ARRIVAL','Renovate','SEALED','OFFERSBRAND','EXPANDABLE','FRIDAY','FLASH','LIMTED'
]

# Create a regex pattern to match any of the words in the list
pattern = r'\b(?:' + '|'.join(map(re.escape, words_to_remove)) + r')\b'

# Apply the regex pattern to remove the words
dataframe["phones_name"] = dataframe["phones_name"].str.replace(pattern, '', regex=True).str.strip()

# Display the cleaned DataFrame
print(dataframe.head())


                                                                  phones_name  \
Unnamed: 0 phones_name                                         phones_reviews   
0          Itel S23 Plus 128GB + 8(4+4)GB RAM 6.78'' Big C...             178   
1          Refurbished OPPO F11 A9 Smart Phone 128GB+6GB 6...              24   
2          XIAOMI Redmi 14C 128GB Storage Upto 8(4+4)GB Ra...             157   
3          New Arrival TECNO POP 9 128GB + 6(3+3)GB 4.5G F...              95   

                                                                unknown_col  \
Unnamed: 0 phones_name                                         phones_price   
0          Itel S23 Plus 128GB + 8(4+4)GB RAM 6.78'' Big C...        13,599   
1          Refurbished OPPO F11 A9 Smart Phone 128GB+6GB 6...         7,499   
2          XIAOMI Redmi 14C 128GB Storage Upto 8(4+4)GB Ra...        12,910   
3          New Arrival TECNO POP 9 128GB + 6(3+3)GB 4.5G F...        11,499   

                                      

In [63]:
dataframe.to_csv("kilimall_clean_phones.csv", index=False)

In [64]:

# Function to clean the phones_name column
def clean_phones_name(name):
    # Remove unwanted characters: (), {}, !!, [], "", and numbers before the first letter
    name = re.sub(r'[\(\)\{\}\[\]\"!]+', '', name)  # Remove parentheses, brackets, quotes, and exclamation marks
    name = re.sub(r'^\d+\s*', '', name)  # Remove numbers at the start of the string
    return name.strip()  # Remove leading and trailing whitespace

# Apply the cleaning function to the phones_name column
dataframe["phones_name"] = dataframe["phones_name"].apply(clean_phones_name)

# Display the cleaned DataFrame
print(dataframe.head())



                                                                  phones_name  \
Unnamed: 0 phones_name                                         phones_reviews   
0          Itel S23 Plus 128GB + 8(4+4)GB RAM 6.78'' Big C...                   
1          Refurbished OPPO F11 A9 Smart Phone 128GB+6GB 6...                   
2          XIAOMI Redmi 14C 128GB Storage Upto 8(4+4)GB Ra...                   
3          New Arrival TECNO POP 9 128GB + 6(3+3)GB 4.5G F...                   

                                                                unknown_col  \
Unnamed: 0 phones_name                                         phones_price   
0          Itel S23 Plus 128GB + 8(4+4)GB RAM 6.78'' Big C...        13,599   
1          Refurbished OPPO F11 A9 Smart Phone 128GB+6GB 6...         7,499   
2          XIAOMI Redmi 14C 128GB Storage Upto 8(4+4)GB Ra...        12,910   
3          New Arrival TECNO POP 9 128GB + 6(3+3)GB 4.5G F...        11,499   

                                      

In [65]:

# Function to remove the first quotation mark
def remove_first_quote(name):
    return re.sub(r'^[\'"]', '', name)  # Remove the first single or double quote if present

# Apply the function to the phones_name column
dataframe["phones_name"] = dataframe["phones_name"].apply(remove_first_quote)

# Display the cleaned DataFrame
print(dataframe.head())

dataframe.to_csv("kilimall_clean_phones.csv", index=False)

                                                                  phones_name  \
Unnamed: 0 phones_name                                         phones_reviews   
0          Itel S23 Plus 128GB + 8(4+4)GB RAM 6.78'' Big C...                   
1          Refurbished OPPO F11 A9 Smart Phone 128GB+6GB 6...                   
2          XIAOMI Redmi 14C 128GB Storage Upto 8(4+4)GB Ra...                   
3          New Arrival TECNO POP 9 128GB + 6(3+3)GB 4.5G F...                   

                                                                unknown_col  \
Unnamed: 0 phones_name                                         phones_price   
0          Itel S23 Plus 128GB + 8(4+4)GB RAM 6.78'' Big C...        13,599   
1          Refurbished OPPO F11 A9 Smart Phone 128GB+6GB 6...         7,499   
2          XIAOMI Redmi 14C 128GB Storage Upto 8(4+4)GB Ra...        12,910   
3          New Arrival TECNO POP 9 128GB + 6(3+3)GB 4.5G F...        11,499   

                                      

In [66]:

# Function to ensure the first character is an alphabet
def remove_non_alphabet_start(name):
    return re.sub(r'^[^a-zA-Z]+', '', name)  # Remove any non-alphabet characters from the start

# Apply the function to the phones_name column
dataframe["phones_name"] = dataframe["phones_name"].apply(remove_non_alphabet_start)

# Display the cleaned DataFrame
print(dataframe.head())
dataframe.to_csv("kilimall_clean_phones.csv", index=False)


                                                                  phones_name  \
Unnamed: 0 phones_name                                         phones_reviews   
0          Itel S23 Plus 128GB + 8(4+4)GB RAM 6.78'' Big C...                   
1          Refurbished OPPO F11 A9 Smart Phone 128GB+6GB 6...                   
2          XIAOMI Redmi 14C 128GB Storage Upto 8(4+4)GB Ra...                   
3          New Arrival TECNO POP 9 128GB + 6(3+3)GB 4.5G F...                   

                                                                unknown_col  \
Unnamed: 0 phones_name                                         phones_price   
0          Itel S23 Plus 128GB + 8(4+4)GB RAM 6.78'' Big C...        13,599   
1          Refurbished OPPO F11 A9 Smart Phone 128GB+6GB 6...         7,499   
2          XIAOMI Redmi 14C 128GB Storage Upto 8(4+4)GB Ra...        12,910   
3          New Arrival TECNO POP 9 128GB + 6(3+3)GB 4.5G F...        11,499   

                                      

In [68]:
ram_pattern = r'\b(2|3|4|6|8|12|24)\s*GB'

# Function to extract RAM size
def extract_ram(phones_name):
    ram = re.findall(ram_pattern, phones_name)  # Find all occurrences that match the pattern
    return ram[0] if ram else None  # If found, return the first match; else return None

storage_pattern = r'\b(16|32|64|128|256)\s*GB'
def extract_storage(phones_name):
    storage = re.findall(storage_pattern, phones_name)  # Find all occurrences that match the pattern
    return storage[0] if storage else None  # If found, return the first match; else return None
def extract_brand(phones_name):
    # Split the name into words and take the first two or three
    words = phones_name.split()
    return ' '.join(words[:3])  # Return the first three words (or fewer if not available)
def extract_battery(phones_name):
    # Search for a pattern that captures numbers followed by 'mAh'
    match = re.search(r'(\d+)\s*mAh', phones_name)
    if match:
        return match.group(1)  # Return the first captured group (the number)
    return None  # If no match, return None

# Apply the function to the 'phones_name' column and create a new 'battery' column
dataframe['battery'] = dataframe['phones_name'].apply(extract_battery)
# Apply the function to the 'phones_name' column and create a new 'storage' column
dataframe['ram'] = dataframe['phones_name'].apply(extract_ram)
dataframe['storage'] = dataframe['phones_name'].apply(extract_storage)
# Apply the function to the 'phones_name' column and create a new 'brand' column
dataframe['brand'] = dataframe['phones_name'].apply(extract_brand)
dataframe["descriptions"] = dataframe['phones_name']
dataframe['source'] = ['kilimall'] *len(dataframe)
dataframe['id'] = [i for i in range(1,len(dataframe)+1)]
dataframe['urls'] = dataframe['phone_url']
dataframe.drop(columns = ['phones_name'])

KeyError: 'phone_url'

In [50]:
dataframe.columns

Index(['phones_name', 'unknown_col', 'phones_price', 'battery', 'ram',
       'storage', 'brand', 'descriptions', 'source', 'id'],
      dtype='object')

In [45]:


dataframe.to_csv("kilimall_clean_phones.csv", index=False)

In [23]:

dataframe['brand'] = dataframe['phones_name'].apply(extract_brand)
dataframe['storage'] = dataframe['phones_name'].apply(extract_storage)
dataframe['ram'] = dataframe['phones_name'].apply(extract_ram)
dataframe['battery'] = dataframe['phones_name'].apply(extract_battery)
dataframe.rename(columns={'unknown_col': 'reviews'}, inplace=True)

new_dataframe = dataframe[['brand', 'storage', 'ram', 'phones_price', 'reviews', 'battery']]

# Display the updated DataFrame
print(new_dataframe)




                                                                                     brand  \
Unnamed: 0 phones_name                                        phones_reviews  phones_price   
0          Itel S23 Plus 128GB + 8(4+4)GB RAM 6.78'' Big C... 178                            
1          Refurbished OPPO F11 A9 Smart Phone 128GB+6GB 6... 24                             
2          XIAOMI Redmi 14C 128GB Storage Upto 8(4+4)GB Ra... 157                            
3          New Arrival TECNO POP 9 128GB + 6(3+3)GB 4.5G F... 94                             
...                                                                                    ...   
5359       Samsung Galaxy A16, 6.5''HD+, 6GB RAM + 128GB R... 0                              
5360       VILLAON V20 SE 6.3" 2GB RAM+32GB Storage 8MP Ca... 0                              
5361       XIAOMI Redmi Note 13, 6.67'', 8GBRAM+ 256GBROM,... 0                              
5362       GALAXY A06,6GB RAM & 128GB ROM, & 5000MAH,DUAL ..

In [25]:

new_dataframe.to_csv("kilimall_clean_phones.csv", index=False)

In [26]:
new_dataframe.rename(columns ={'phone_price':'price'}, inplace=True)
new_dataframe.to_csv("kilimall_clean_phones.csv", index=False)

C:\Users\Vivian.Obino1\AppData\Local\Temp\ipykernel_16352\342716474.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_dataframe.rename(columns ={'phone_price':'price'}, inplace=True)
